In [6]:
# data_loader.py
import psycopg2
# from fetch_graphql import fetch_products_df,fetch_stock_df, fetch_orders_df
import streamlit as st
import json
import gspread
import pandas as pd
import requests
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials

# load_dotenv(dotenv_path='config.env')  # 👈 important ici

In [7]:
SHOP_NAME = st.secrets["SHOP_NAME_TEC"]
ACCESS_TOKEN = st.secrets["SHOPIFY_API_TOKEN"]
GRAPHQL_URL = f"https://{SHOP_NAME}/admin/api/2024-01/graphql.json"
SHOP_NAME_LOCAL = st.secrets["SHOP_NAME_LOCAL"]

HEADERS = {
    "X-Shopify-Access-Token": ACCESS_TOKEN,
    "Content-Type": "application/json"
}

GRAPHQL_URL = f"https://{SHOP_NAME}/admin/api/2024-04/graphql.json"

BASE_URL = f"https://{SHOP_NAME_LOCAL}/admin/api/2024-04"

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]
json_creds = json.loads(st.secrets["GCP_SERVICE_ACCOUNT"])
creds = ServiceAccountCredentials.from_json_keyfile_dict(json_creds, scope)
client = gspread.authorize(creds)


In [8]:
# Simple requête test GraphQL
query = """
{
  shop {
    name
    myshopifyDomain
    email
  }
}
"""

res = requests.post(
    f"https://{SHOP_NAME}/admin/api/2024-04/graphql.json",
    headers=HEADERS,
    json={"query": query}
)

print(res.status_code)
print(res.json())

200
{'data': {'shop': {'name': 'Local', 'myshopifyDomain': 'fh1z1f-5i.myshopify.com', 'email': 'info@seulementlocal.com'}}, 'extensions': {'cost': {'requestedQueryCost': 1, 'actualQueryCost': 1, 'throttleStatus': {'maximumAvailable': 2000.0, 'currentlyAvailable': 1999, 'restoreRate': 100.0}}}}


In [11]:
# ----------------------------------------
# 1. PRODUCTS_DF
# ----------------------------------------
QUERY_PRODUCTS = """
query getProductsAndVariants($cursor: String) {
  products(first: 100, after: $cursor) {
    pageInfo { hasNextPage }
    edges {
      cursor
      node {
        id
        title
        vendor
        productType
        handle
        updatedAt
        tags
        publishedAt
        bodyHtml
        images(first: 1) {
          edges {
            node {
              id
              src
            }
          }
        }
        variants(first: 100) {
          edges {
            node {
              id
              title
              sku
              price
              option1
              option2
              option3
              updatedAt
              inventoryItem {
                id
                inventoryQuantity
              }
              image {
                id
              }
            }
          }
        }
      }
    }
  }
}
"""

def fetch_products_df():
    cursor, has_next = None, True
    rows = []
    while has_next:
        r = requests.post(GRAPHQL_URL, headers=HEADERS, json={"query": QUERY_PRODUCTS, "variables": {"cursor": cursor}})
        r.raise_for_status()
        data = r.json()["data"]["products"]
        
        products = data
        for edge in products["edges"]:
            cursor = edge["cursor"]
            product = edge["node"]
            for v_edge in product["variants"]["edges"]:
                variant = v_edge["node"]
                rows.append({
                    "product_id": product["id"],
                    "title": product["title"],
                    "vendor": product["vendor"],
                    "product_type": product["productType"],
                    "tags": ", ".join(product["tags"]),
                    "handle": product["handle"],
                    "product_updated_at": product["updatedAt"],
                    "published_at": product["publishedAt"],
                    "body_html": product["bodyHtml"],
                    "variant_id": variant["id"],
                    "variant_title": variant["title"],
                    "sku": variant["sku"],
                    "price": variant["price"],
                    "option1": variant["option1"],
                    "option2": variant["option2"],
                    "option3": variant["option3"],
                    "variant_updated_at": variant["updatedAt"],
                    "image_id": variant["image"]["id"] if variant["image"] else None,
                    "inventory_item_id": variant["inventoryItem"]["id"],
                    "inventory_quantity": variant["inventoryItem"]["inventoryQuantity"],
                })
        has_next = products["pageInfo"]["hasNextPage"]

    return pd.DataFrame(rows)

In [12]:
fetch_products_df()

KeyError: 'data'

In [57]:
# ----------------------------------------
# 2. STOCK_DF
# ----------------------------------------
QUERY_STOCK = """
query getInventoryLevels($cursor: String) {
  products(first: 100, after: $cursor) {
    pageInfo { hasNextPage }
    edges {
      cursor
      node {
        id
        title
        variants(first: 100) {
          edges {
            node {
              id
              sku
              inventoryItem {
                id
                sku
                tracked
                createdAt
                updatedAt
                requiresShipping
                countryOfOrigin
                provinceOfOrigin
                inventoryLevels(first: 5) {
                  edges {
                    node {
                      availableQuantity
                      updatedAt
                      location {
                        id
                        name
                      }
                    }
                  }
                }
              }
            }
          }
        }
      }
    }
  }
}
"""

def fetch_stock_df():
    cursor, has_next = None, True
    rows = []
    while has_next:
        r = requests.post(GRAPHQL_URL, headers=HEADERS, json={"query": QUERY_STOCK, "variables": {"cursor": cursor}})
        r.raise_for_status()
        data = r.json()["data"]["products"]
        for edge in data["edges"]:
            cursor = edge["cursor"]
            for v_edge in edge["node"]["variants"]["edges"]:
                variant = v_edge["node"]
                inv = variant.get("inventoryItem")
                if inv and inv.get("tracked"):
                    for lvl in inv.get("inventoryLevels", {}).get("edges", []):
                        node = lvl["node"]
                        qty = next((q["quantity"] for q in node.get("quantities", []) if q["name"] == "available"), None)
                        rows.append({
                            "variant_id": variant["id"],
                            "sku": variant["sku"],
                            "location": node["location"]["name"],
                            "available": qty
                        })
        has_next = data["pageInfo"]["hasNextPage"]
    return pd.DataFrame(rows)

In [58]:
# ----------------------------------------
# 3. ORDERS_DF
# ----------------------------------------
QUERY_ORDERS = """
query getOrdersWithLineItems($cursor: String) {
  orders(first: 100, after: $cursor, reverse: true) {
    pageInfo { hasNextPage }
    edges {
      cursor
      node {
        id
        name
        processedAt: createdAt
        email
        financialStatus
        fulfillmentStatus
        tags
        customer {
          id
          ordersCount: ordersCount
          totalSpent
          tags
          firstName
          lastName
        }
        lineItems(first: 50) {
          edges {
            node {
              id
              title
              name
              quantity
              sku
              variantTitle
              vendor
              fulfillmentService
              productId
              requiresShipping
              taxable
              giftCard
              properties {
                name
                value
              }
              productExists
              fulfillableQuantity
              fulfillmentStatus
              originalUnitPrice
              discountedUnitPrice
            }
          }
        }
      }
    }
  }
}
"""

def fetch_orders_df():
    cursor, has_next = None, True
    rows = []
    while has_next:
        r = requests.post(GRAPHQL_URL, headers=HEADERS, json={"query": QUERY_ORDERS, "variables": {"cursor": cursor}})
        r.raise_for_status()
        data = r.json()["data"]["orders"]
        for edge in data["edges"]:
            cursor = edge["cursor"]
            o = edge["node"]
            for li in o["lineItems"]["edges"]:
                item = li["node"]
                rows.append({
                    "order_id": o["id"],
                    "order_name": o["name"],
                    "created_at": o["createdAt"],
                    "title": item["title"],
                    "sku": item["sku"],
                    "quantity": item["quantity"],
                    "original_unit_price": float(item["originalUnitPrice"]),
                    "discounted_unit_price": float(item["discountedUnitPrice"])
                })
        has_next = data["pageInfo"]["hasNextPage"]
    return pd.DataFrame(rows)

In [75]:
# https://docs.google.com/spreadsheets/d/1NATO1x_KlDUQ6F0vbD32xqM_hPsS0NqDawo3_CdcO0o/edit?gid=1977368994#gid=1977368994
# ----------------------------------------
# 3. VENDORS_SPEC_DF -> GOOGLE SHEETS
# ----------------------------------------
def fetch_vendors_df():
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("gspread_key.json", scope)
    client = gspread.authorize(creds)

    # Ouvre le Google Sheet avec l’ID du nouveau fichier
    sheet = client.open_by_key("1NATO1x_KlDUQ6F0vbD32xqM_hPsS0NqDawo3_CdcO0o")

    # Onglet cible à modifier si besoin
    worksheet = sheet.worksheet("Business_rules")

    # Transforme en DataFrame
    df = get_as_dataframe(worksheet, evaluate_formulas=True)
    df = df.dropna(how="all")  # Enlève les lignes vides
    df.columns = [c.strip() for c in df.columns]  # Nettoie les noms de colonnes
    return df

In [77]:
# ----------------------------------------
# 4. FETCH ALL TOGETHER
# ----------------------------------------
def fetch_datasets():
    products_df = fetch_products_df()
    stock_df = fetch_stock_df()
    orders_df = fetch_orders_df()
    vendors_df = fetch_vendors_df()
    return products_df, stock_df, orders_df, vendors_df

# Run it
products_df, stock_df, orders_df,vendors_df = fetch_datasets()

In [78]:
print("🔹 Products:", products_df.shape, "rows")
print("🔹 Stockm:", stock_df.shape, "rows")
print("🔹 Orders:", orders_df.shape, "rows")
print("🔹 Vendors:", vendors_df.shape, "rows")

🔹 Products: (790, 8) rows
🔹 Stockm: (790, 4) rows
🔹 Orders: (727, 8) rows
🔹 Vendors: (13, 8) rows


In [80]:
products_df.to_csv("products.csv", index=False)
stock_df.to_csv("stock.csv", index=False)
orders_df.to_csv("orders.csv", index=False)
vendors_df.to_csv("vendors.csv", index=False)

In [ ]:
## CLEAN SKU

In [15]:
import requests
import re
import time

# Shopify API setup
GRAPHQL_URL = f"https://{SHOP_NAME}/admin/api/2024-01/graphql.json"
HEADERS = {
    "Content-Type": "application/json",
    "X-Shopify-Access-Token": ACCESS_TOKEN
}

def slugify(text):
    text = re.sub(r"[^\w]+", "-", text.upper())
    return text[:12]

def generate_sku(product, variant):
    vendor = slugify(product.get("vendor", ""))
    ptype = slugify(product.get("productType", ""))
    title = slugify(product.get("title", ""))
    suffix = variant["id"][-6:]
    return f"{vendor}-{ptype}-{title}-{suffix}"

def update_sku_on_shopify(variant_id, new_sku):
    mutation = """
    mutation updateProductVariant($input: ProductVariantInput!) {
      productVariantUpdate(input: $input) {
        productVariant {
          id
          sku
        }
        userErrors {
          field
          message
        }
      }
    }
    """
    variables = {"input": {"id": variant_id, "sku": new_sku}}
    res = requests.post(GRAPHQL_URL, headers=HEADERS, json={"query": mutation, "variables": variables})
    res.raise_for_status()
    data = res.json()
    errors = data.get("data", {}).get("productVariantUpdate", {}).get("userErrors", [])
    if errors:
        print(f"❌ {variant_id} → {new_sku} : {errors}")
    else:
        print(f"✅ {variant_id} → {new_sku}")

def process_all_products():
    query = """
    query getAllProducts($cursor: String) {
      products(first: 100, after: $cursor) {
        pageInfo {
          hasNextPage
        }
        edges {
          cursor
          node {
            id
            title
            vendor
            productType
            variants(first: 100) {
              edges {
                node {
                  id
                  sku
                }
              }
            }
          }
        }
      }
    }
    """
    cursor = None
    has_next = True

    while has_next:
        res = requests.post(GRAPHQL_URL, headers=HEADERS, json={"query": query, "variables": {"cursor": cursor}})
        res.raise_for_status()
        data = res.json()["data"]["products"]
        for edge in data["edges"]:
            cursor = edge["cursor"]
            product = edge["node"]
            for v_edge in product["variants"]["edges"]:
                variant = v_edge["node"]
                new_sku = generate_sku(product, variant)
                update_sku_on_shopify(variant["id"], new_sku)
                time.sleep(0.5)  # limiter les appels pour éviter rate limit
        has_next = data["pageInfo"]["hasNextPage"]

process_all_products()


✅ gid://shopify/ProductVariant/44663151820998 → GIRL-CRUSH-CREWNECK-PULL-RENDEZ--820998
✅ gid://shopify/ProductVariant/44663151853766 → GIRL-CRUSH-CREWNECK-PULL-RENDEZ--853766
✅ gid://shopify/ProductVariant/44663151886534 → GIRL-CRUSH-CREWNECK-PULL-RENDEZ--886534
✅ gid://shopify/ProductVariant/44663151919302 → GIRL-CRUSH-CREWNECK-PULL-RENDEZ--919302
✅ gid://shopify/ProductVariant/44663151952070 → GIRL-CRUSH-CREWNECK-PULL-RENDEZ--952070
✅ gid://shopify/ProductVariant/44663151984838 → GIRL-CRUSH-CREWNECK-PULL-RENDEZ--984838
✅ gid://shopify/ProductVariant/44663152017606 → GIRL-CRUSH-CREWNECK-PULL-RENDEZ--017606
✅ gid://shopify/ProductVariant/44663152050374 → GIRL-CRUSH-ACCESSOIRES-TOTE-BAG-EN--050374
✅ gid://shopify/ProductVariant/44663152148678 → GIRL-CRUSH-ACCESSOIRES-JOURNAL-WORK-148678
✅ gid://shopify/ProductVariant/44663152312518 → GIRL-CRUSH-CREWNECK-CREWNECK-ESS-312518
✅ gid://shopify/ProductVariant/44663152345286 → GIRL-CRUSH-CREWNECK-CREWNECK-ESS-345286
✅ gid://shopify/ProductVar